# 1) Import packages and data

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
# get data from drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [108]:
df = pd.read_csv('Cocoon_official_15042023.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3826 entries, 0 to 3825
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           3694 non-null   object
 1   Price          3619 non-null   object
 2   Sales          3661 non-null   object
 3   URL            3826 non-null   object
 4   Rating         3586 non-null   object
 5   Rating record  3574 non-null   object
 6   Date           3826 non-null   object
 7   Discount       563 non-null    object
dtypes: object(8)
memory usage: 239.2+ KB


# 2) Simple processing on null and unnecessary data

1) Eliminate 'URL' column as I will not be needing it in the future

In [109]:
df=df.drop(columns=['URL'])

2) Filter to eliminate null value in "Name". For sales data, later on, it will use filter code which is only aplicaple with nonnull data (step 4)

In [7]:
df = df.dropna(subset=['Name', 'Sales'])

In [100]:
df = df.dropna(subset=['Name'])

In [8]:
print(df)

                                                   Name     Price  \
0     HÀNG TẶNG KHÔNG BÁN - Gel rửa mặt cà phê Đắk L...   195.000   
1     Gel rửa mặt cà phê Đắk Lắk Cocoon cho làn da t...   195.000   
2     [DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Bơ dưỡng ...   340.000   
3     Combo Sáng Mịn Sạch Mụn: 1 Gel tắm bí đao Coco...   351.000   
4     [Mã COSCN12 giảm 8%] Bơ dưỡng thể cà phê Đắk L...   215.000   
...                                                 ...       ...   
3821  [Mã COSCN12 giảm 8%] Bigsize - Nước tẩy trang ...  ₫250.750   
3822  [Mã COSCN12 giảm 8%] Bigsize - Thạch hoa hồng ...  ₫327.250   
3823  [Mã COSCN12 giảm 8%] Nước hoa hồng (toner) Coc...  ₫165.750   
3824  [Mã COSCN12 giảm 8%] Tinh chất hoa hồng (serum...  ₫225.250   
3825  [Mã COSCN12 giảm 8%] Thạch hoa hồng dưỡng ẩm (...  ₫165.750   

             Sales Rating Rating record       Date  Discount  
0                     NaN           NaN   2/9/2023       NaN  
1       Đã bán 175    4.9            64   2/9

In [9]:
# check if df reach needed non-null features and check for type of data for the next step
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3661 entries, 0 to 3825
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           3661 non-null   object
 1   Price          3586 non-null   object
 2   Sales          3661 non-null   object
 3   Rating         3583 non-null   object
 4   Rating record  3571 non-null   object
 5   Date           3661 non-null   object
 6   Discount       534 non-null    object
dtypes: object(7)
memory usage: 228.8+ KB


Name                0
Price              75
Sales               0
Rating             78
Rating record      90
Date                0
Discount         3127
dtype: int64

# 3) Date

In [101]:
from datetime import datetime
# filter dataframe into two subset right_dates and error_dates, which are date in the right format "mm/dd/yyyy" and those aren't 
e_dates = pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='coerce').isna()
error_dates = df[e_dates]

r_dates = pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='coerce').notna()
right_dates = df[r_dates]

#change date in wrong format to right format
error_dates['Date'] = pd.to_datetime(error_dates['Date'], format='%d/%m/%Y').dt.strftime('%m/%d/%Y')

#combine subset back to one original one
df = pd.concat([error_dates, right_dates], ignore_index=True)

# 4) Sales

4.1) Remove "Da ban" part in "Sales" and  to make "Sales" in the same structure.

In [14]:
da_ban_data = df[df['Sales'].str.startswith('Đã bán')]
non_da_ban_data = df[~df['Sales'].str.startswith('Đã bán')]
# remove "Đã bán" part from beginning of each string
da_ban_data['Sales'] = da_ban_data['Sales'].str.replace('Đã bán', '')
# merge filtered dataset back into original dataset
df = pd.concat([da_ban_data, non_da_ban_data], ignore_index=True)

4.2) turn "k" to thousands and change data type (to make "Sales" in the same type and structure)

In [15]:
def convert_to_numeric(strings):
    if "k" in strings:
        num = float(strings.replace("k", "").replace(',', '.'))*1000
        return num
    else:
        return strings
df['Sales'] = df['Sales'].apply(convert_to_numeric)
# there are data that is empty string as initially it can be "Da ban" with no sale number
# therefore, replace those with a valid value or remove the row entirely and change column type to numeric
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce').fillna(0)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3661 entries, 0 to 3660
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           3661 non-null   object        
 1   Price          3586 non-null   object        
 2   Sales          3661 non-null   float64       
 3   Rating         3583 non-null   object        
 4   Rating record  3571 non-null   object        
 5   Date           3660 non-null   datetime64[ns]
 6   Discount       534 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 200.3+ KB


In [17]:
# print the names of the products with sales equal to 0
zero_sales = df[df["Sales"] == 0.0]
zero_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 234 to 2906
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           60 non-null     object        
 1   Price          60 non-null     object        
 2   Sales          60 non-null     float64       
 3   Rating         25 non-null     object        
 4   Rating record  24 non-null     object        
 5   Date           60 non-null     datetime64[ns]
 6   Discount       6 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 3.8+ KB


# 5) Rating and Rating record


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3661 entries, 0 to 3660
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           3661 non-null   object        
 1   Price          3585 non-null   float64       
 2   Sales          3661 non-null   float64       
 3   Rating         3583 non-null   object        
 4   Rating record  3571 non-null   object        
 5   Date           3660 non-null   datetime64[ns]
 6   Discount(%)    533 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 200.3+ KB


In [42]:
df['Rating record'] = df['Rating record'].fillna(value='0')

In [43]:
def convert_to_numeric(strings):
    if "k" in strings:
        num = float(strings.replace("k", "").replace(',', '.'))*1000
        return num
    else:
        return strings

In [44]:
df['Rating record'] = df['Rating record'].apply(convert_to_numeric)
# there are data that is empty string as initially it can be "Da ban" with no sale number
# therefore, replace those with a valid value or remove the row entirely and change column type to numeric
df['Rating record'] = pd.to_numeric(df['Rating record'], errors='coerce').fillna(0)

In [45]:
df

,Name,Price,Sales,Rating,Rating record,Date,Discount(%)
0,Gel rửa mặt cà phê Đắk Lắk Cocoon cho làn da t...,195.00,175.0,4.9,64.0,2023-02-09,NaN
1,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Bơ dưỡng ...,340.00,629.0,,0.0,2023-02-09,NaN
2,Combo Sáng Mịn Sạch Mụn: 1 Gel tắm bí đao Coco...,351.00,1400.0,NaN,0.0,2023-02-09,NaN
3,[Mã COSCN12 giảm 8%] Bơ dưỡng thể cà phê Đắk L...,215.00,10500.0,5.0,4300.0,2023-02-09,NaN
4,[Mã COSCN12 giảm 8%] Combo làm sạch da chết và...,323.00,1600.0,5.0,576.0,2023-02-09,NaN
...,...,...,...,...,...,...,...
3656,[Mã COSCN12 giảm 8%] Bigsize - Nước tẩy trang ...,250.75,8900.0,"4,9",3200.0,2023-04-15,15.0
3657,[Mã COSCN12 giảm 8%] Bigsize - Thạch hoa hồng ...,327.25,6900.0,"4,9",2500.0,2023-04-15,15.0
3658,[Mã COSCN12 giảm 8%] Nước hoa hồng (toner) Coc...,165.75,13100.0,5,4800.0,2023-04-15,15.0
3659,[Mã COSCN12 giảm 8%] Tinh chất hoa hồng (serum...,225.25,4000.0,5,1500.0,2023-04-15,15.0


In [56]:
df['Rating'] = df['Rating'].str.replace('Chưa Có Đánh Giá', '0')

# therefore, replace those with a valid value or remove the row entirely and change column type to numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)
df = df.dropna(subset=['Rating'])

# 6) Price

In [19]:
# extract only the numeric part of the "Price" column
df["Price"] = df["Price"].str.extract(r"(\d+\.?\d*)").astype(float)

In [20]:
# extract only the numeric part of the "Discount" column
df["Discount"] = df["Discount"].str.extract(r"(\d+\.?\d*)").astype(float)
df.columns = df.columns.str.replace('Discount', 'Discount(%)')

# 7) Category and Product ID

In [46]:
# define a function to assign categories based on product names
def assign_category(name):
    if "cà phê" in name.lower():
        return "Coffee"
    elif "hoa hồng" in name.lower():
        return "Rose"
    elif "bưởi" in name.lower():
        return "Pomelo"
    elif "nghệ" in name.lower():
        return "Ginger"
    elif "bí đao" in name.lower():
        return "Squash"
    else:
        return "Other"

# define a function to assign categories based on product names
def assign_ini(name):
    if "cà phê" in name.lower():
        return "C"
    elif "hoa hồng" in name.lower():
        return "R"
    elif "bưởi" in name.lower():
        return "P"
    elif "nghệ" in name.lower():
        return "G"
    elif "bí đao" in name.lower():
        return "S"
    else:
        return "O"
# apply the function to create a new "category" column
df["Category"] = df["Name"].apply(lambda name: assign_category(name))
df["Ini_cat"] = df["Name"].apply(lambda name: assign_ini(name))

In [47]:
#create seperate dataset for each category
coffee_df = df[df["Category"] == "Coffee"]
rose_df = df[df["Category"] == "Rose"]
pomelo_df = df[df["Category"] == "Pomelo"]
ginger_df = df[df["Category"] == "Ginger"]
squash_df = df[df["Category"] == "Squash"]

# group by product name and assign a product ID to each category
coffee_df["Group_ID"] = coffee_df.groupby("Name").ngroup() + 1
rose_df["Group_ID"] = rose_df.groupby("Name").ngroup() + 1
pomelo_df["Group_ID"] = pomelo_df.groupby("Name").ngroup() + 1
ginger_df["Group_ID"] = ginger_df.groupby("Name").ngroup() + 1
squash_df["Group_ID"] = squash_df.groupby("Name").ngroup() + 1

In [48]:
# merge data with category type and ID back as official dataset 
df=pd.concat([coffee_df, rose_df, pomelo_df, ginger_df, squash_df], ignore_index=True)

# create Product_ID column based on category initial and category ID
df['Product_ID'] = df.apply(lambda row: str(row['Ini_cat']) + str(row['Group_ID']), axis=1)

In [49]:
discount_program = df[df["Discount(%)"].notna()]
discount_program

,Name,Price,Sales,Rating,Rating record,Date,Discount(%),Category,Ini_cat,Group_ID,Product_ID
268,HÀNG TẶNG KHÔNG BÁN - Tẩy da chết mặt cà phê Đ...,165.00,3600.0,5,1300.0,2023-02-28,15.0,Coffee,C,8,C8
269,[Mã COSCN12 giảm 8%] Combo làm sạch da chết và...,323.00,1600.0,5,589.0,2023-02-28,15.0,Coffee,C,19,C19
270,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Bơ dưỡng ...,340.00,734.0,"4,9",226.0,2023-02-28,15.0,Coffee,C,16,C16
271,[Mã COSCN12 giảm 8%] Tẩy da chết môi cà phê Đắ...,75.00,20400.0,5,8200.0,2023-02-28,15.0,Coffee,C,21,C21
273,[Mã COSCN12 giảm 8%] Tẩy da chết cơ thể cà phê...,125.00,121200.0,5,44700.0,2023-02-28,15.0,Coffee,C,20,C20
...,...,...,...,...,...,...,...,...,...,...,...
3656,[COMBO MUA 1 TẶNG 1 ÁP DỤNG 01/04-30/04] Kem c...,395.00,621.0,"4,9",172.0,2023-04-15,43.0,Squash,S,20,S20
3657,[Mã COSCN12 giảm 8%] Bigsize - Nước bí đao cân...,250.75,5500.0,"4,9",2000.0,2023-04-15,15.0,Squash,S,24,S24
3658,Tinh chất bí đao (serum) 7% Niacinamide sạch m...,250.75,4000.0,"4,9",1400.0,2023-04-15,15.0,Squash,S,18,S18
3659,[Mã COSCN12 giảm 8%] Nước bí đao cân bằng da (...,165.75,17600.0,5,7400.0,2023-04-15,15.0,Squash,S,33,S33


In [50]:
# Final check for result of preprocessing data: 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3661 entries, 0 to 3660
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           3661 non-null   object        
 1   Price          3585 non-null   float64       
 2   Sales          3661 non-null   float64       
 3   Rating         3583 non-null   object        
 4   Rating record  3661 non-null   float64       
 5   Date           3660 non-null   datetime64[ns]
 6   Discount(%)    533 non-null    float64       
 7   Category       3661 non-null   object        
 8   Ini_cat        3661 non-null   object        
 9   Group_ID       3661 non-null   int64         
 10  Product_ID     3661 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 314.7+ KB


In [51]:
# Record information of the final data set as "final_data_info" 
final_data_info= df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3661 entries, 0 to 3660
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Name           3661 non-null   object        
 1   Price          3585 non-null   float64       
 2   Sales          3661 non-null   float64       
 3   Rating         3583 non-null   object        
 4   Rating record  3661 non-null   float64       
 5   Date           3660 non-null   datetime64[ns]
 6   Discount(%)    533 non-null    float64       
 7   Category       3661 non-null   object        
 8   Ini_cat        3661 non-null   object        
 9   Group_ID       3661 non-null   int64         
 10  Product_ID     3661 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 314.7+ KB


In [52]:
# Write the DataFrame to a CSV file
df.to_csv("Preprocessed_data.csv", index=True)
coffee_df.to_csv("Coffee_data.csv", index=True)
rose_df.to_csv("Rose_df.csv", index=True)
pomelo_df.to_csv("Pomelo_df.csv", index=True)
ginger_df.to_csv("Ginger_df.csv", index=True)
squash_df.to_csv("Squash_df.csv", index=True)

In [53]:
df

,Name,Price,Sales,Rating,Rating record,Date,Discount(%),Category,Ini_cat,Group_ID,Product_ID
0,Gel rửa mặt cà phê Đắk Lắk Cocoon cho làn da t...,195.00,175.0,4.9,64.0,2023-02-09,NaN,Coffee,C,4,C4
1,[DEAL 1 TẶNG 1 ĐỘC QUYỀN FLASH SALE] Bơ dưỡng ...,340.00,629.0,,0.0,2023-02-09,NaN,Coffee,C,16,C16
2,Combo Sáng Mịn Sạch Mụn: 1 Gel tắm bí đao Coco...,351.00,1400.0,NaN,0.0,2023-02-09,NaN,Coffee,C,3,C3
3,[Mã COSCN12 giảm 8%] Bơ dưỡng thể cà phê Đắk L...,215.00,10500.0,5.0,4300.0,2023-02-09,NaN,Coffee,C,17,C17
4,[Mã COSCN12 giảm 8%] Combo làm sạch da chết và...,323.00,1600.0,5.0,576.0,2023-02-09,NaN,Coffee,C,19,C19
...,...,...,...,...,...,...,...,...,...,...,...
3656,[COMBO MUA 1 TẶNG 1 ÁP DỤNG 01/04-30/04] Kem c...,395.00,621.0,"4,9",172.0,2023-04-15,43.0,Squash,S,20,S20
3657,[Mã COSCN12 giảm 8%] Bigsize - Nước bí đao cân...,250.75,5500.0,"4,9",2000.0,2023-04-15,15.0,Squash,S,24,S24
3658,Tinh chất bí đao (serum) 7% Niacinamide sạch m...,250.75,4000.0,"4,9",1400.0,2023-04-15,15.0,Squash,S,18,S18
3659,[Mã COSCN12 giảm 8%] Nước bí đao cân bằng da (...,165.75,17600.0,5,7400.0,2023-04-15,15.0,Squash,S,33,S33


In [57]:
df['Rating'].value_counts()

0.0    2006
5.0    1312
4.9     330
4.0      11
4.7       2
Name: Rating, dtype: int64

# 8) Check data information for final result


In [67]:
df1= pd.read_csv('Preprocessed_data.csv')

In [ ]:
#print(df.describe())
df.describe()

,Name,Price,Sales,URL,Rating,Rating record,Date,Discount
count,3694,3619,3661,3826,3586,3574,3826,563
unique,119,142,763,3012,13,596,68,9
top,Thạch nghệ Hưng Yên giúp làm sáng da và mờ thâ...,₫195.000,"1,1k",https://shopee.vn/-M%C3%A3-COSCN12-gi%E1%BA%A3...,"4,9","1,9k",4/14/2023,15% giảm
freq,68,420,62,4,1790,110,138,433


In [71]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3661 entries, 0 to 3660
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           3661 non-null   object 
 1   Price          3585 non-null   float64
 2   Sales          3661 non-null   object 
 3   Rating         3661 non-null   float64
 4   Rating record  3661 non-null   float64
 5   Date           3660 non-null   object 
 6   Discount(%)    533 non-null    float64
 7   Category       3661 non-null   object 
 8   Ini_cat        3661 non-null   object 
 9   Product_ID     3661 non-null   object 
dtypes: float64(4), object(6)
memory usage: 286.1+ KB


In [ ]:
print(df1.columns)

Index(['Unnamed: 0', 'Name', 'Price', 'Sales', 'Rating', 'Rating record',
       'Date', 'Discount(%)', 'Category', 'Ini_cat', 'Group_ID', 'Product_ID'],
      dtype='object')


In [69]:
df1=df1.drop(columns=['Unnamed: 0','Group_ID'])

In [70]:
# specify a formatting function that converts the scientific notation to a fixed-point notation
pd.options.display.float_format = '{:.2f}'.format
# Rename the columns to include count, std, and mean
stats = df1.describe()
#save result of statiscal dataset information into csv file 
stats.to_csv("Statiscal result.csv", index= True)
stats

,Price,Rating,Rating record,Discount(%)
count,3585.00,3661.00,3661.00,533.00
mean,290.10,2.25,149.41,17.36
std,147.94,2.48,215.42,6.69
min,60.00,0.00,0.00,5.00
25%,195.00,0.00,19.00,15.00
50%,289.00,0.00,46.00,15.00
75%,345.00,5.00,190.00,15.00
max,955.00,5.00,1248.00,48.00
